In [1]:
import csv
import pandas as pd
import numpy as np
import awswrangler as wr
from datetime import datetime, timedelta
import json
import gzip
from io import BytesIO, TextIOWrapper
import boto3
import dask.dataframe as dd
import plotly.express as px
import seaborn as sns

In [2]:
pos_rooftop_day=pd.read_csv("pos_rooftop_day.csv.gz", sep="\t")

In [3]:
pos_rooftop_day.columns

Index(['calendar_date', 'pfcntr', 'dma_name', 'dma_code', 'week_end_sat',
       'ordr_type_name', 'gross_sales_amt', 'net_sales_amt', 'check_cnt'],
      dtype='object')

In [15]:
pos_rooftop_day[pos_rooftop_day.isna()]

,calendar_date,pfcntr,dma_name,dma_code,week_end_sat,ordr_type_name,gross_sales_amt,net_sales_amt,check_cnt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
39923928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39923929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39923930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39923931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pos_missing=pos_rooftop_day[pos_rooftop_day['gross_sales_amt']==0.00].copy()

In [5]:
pos_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4830675 entries, 49 to 39923894
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   calendar_date    object 
 1   pfcntr           int64  
 2   dma_name         object 
 3   dma_code         int64  
 4   week_end_sat     object 
 5   ordr_type_name   object 
 6   gross_sales_amt  float64
 7   net_sales_amt    float64
 8   check_cnt        float64
dtypes: float64(3), int64(2), object(4)
memory usage: 368.6+ MB


In [6]:
pos_missing.drop(columns=['dma_name', 'dma_code'], inplace=True)

In [7]:
pos_missing.head(1000)

,calendar_date,pfcntr,week_end_sat,ordr_type_name,gross_sales_amt,net_sales_amt,check_cnt
49,2018-07-02,300176,2018-07-07,otg_dt,0.0,0.0,0.0
53,2018-07-02,300176,2018-07-07,otg_dt,0.0,0.0,0.0
89,2018-12-13,300176,2018-12-15,otg_dt,0.0,0.0,0.0
93,2018-12-13,300176,2018-12-15,otg_dt,0.0,0.0,0.0
186,2019-10-09,300176,2019-10-12,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...
14643,2020-11-21,300296,2020-11-21,otg_walk_in,0.0,0.0,0.0
14644,2020-11-21,300296,2020-11-21,otg_dt,0.0,0.0,0.0
14647,2020-11-25,300296,2020-11-28,otg_walk_in,0.0,0.0,0.0
14648,2020-11-25,300296,2020-11-28,otg_dt,0.0,0.0,0.0


In [8]:
pos_missing=pd.DataFrame(pos_missing.groupby(['week_end_sat', 'pfcntr']).sum().to_records())

In [9]:
pos_missing

,week_end_sat,pfcntr,gross_sales_amt,net_sales_amt,check_cnt
0,2018-01-06,300178,0.0,0.0,0.0
1,2018-01-06,300202,0.0,0.0,0.0
2,2018-01-06,300205,0.0,0.0,0.0
3,2018-01-06,300208,0.0,0.0,0.0
4,2018-01-06,300215,0.0,0.0,0.0
...,...,...,...,...,...
715206,2021-01-02,359678,0.0,0.0,0.0
715207,2021-01-02,360679,0.0,0.0,0.0
715208,2021-01-02,361797,0.0,0.0,0.0
715209,2021-01-02,361816,0.0,0.0,0.0


In [10]:
pos_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715211 entries, 0 to 715210
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   week_end_sat     715211 non-null  object 
 1   pfcntr           715211 non-null  int32  
 2   gross_sales_amt  715211 non-null  float64
 3   net_sales_amt    715211 non-null  float64
 4   check_cnt        715211 non-null  float64
dtypes: float64(3), int32(1), object(1)
memory usage: 24.6+ MB


In [11]:
pos_missing.to_csv("pos_rooftop_missing.csv", index=False, header=True)

In [12]:
pos_missing

,week_end_sat,pfcntr,gross_sales_amt,net_sales_amt,check_cnt
0,2018-01-06,300178,0.0,0.0,0.0
1,2018-01-06,300202,0.0,0.0,0.0
2,2018-01-06,300205,0.0,0.0,0.0
3,2018-01-06,300208,0.0,0.0,0.0
4,2018-01-06,300215,0.0,0.0,0.0
...,...,...,...,...,...
715206,2021-01-02,359678,0.0,0.0,0.0
715207,2021-01-02,360679,0.0,0.0,0.0
715208,2021-01-02,361797,0.0,0.0,0.0
715209,2021-01-02,361816,0.0,0.0,0.0
